# Ensemble Machine Learning #

# I. Ensemble Learning Methods #
Supervised learning
Group of predictors (Classifiers/regressors)

<b>Bootstrap Aggregating or Bagging</b>
<ul>
<li>Bootstrap sampling: sampling with replacement.</li>
<li>Combine by averaging the output.</li>
<li>Combine by voting.</li>
<li>Can be applied to many classifiers which includes ANN, CART, ...</li>
</ul> 
<b>Pasting</b>
<ul>
<li>Sampling without replacement</li>
</ul>
<b>Boosting </b>
<ul>
<li>Train week Classifiers</li>
<li>Add them to a final strong classifier by weighting. Weighting by accuracy(typically)</li>
<li>Once added, the data is reweighted:</li>
<ul><li>misclassified classes gain weight.</li>
<li> Correctly classified samples lose weight (Exception: boost by majority and Brownboost - decrese the weight of repeatedly misclassified examples)</li>
<li>Algo are forced to learn more from misclassified samples.</li>
</ul></ul>
<b>Stacking</b>
<ul>
<li>Also knownas stacked generalization </li>
<li>From kaggle: combine information from multiple predictive models to generate a new model. Often times  the stacked model (also called 2nd level model) will outperform each of the individual models due its smooting nature and ability to highlight each  base model where it performs best and discredit each base model where it performs poorly. For this reason, stacking is most effective when the base models are significantly different.</li>
<li>Trainin a learning algorithm to combine the predictions of several other learning algorithms.</li>
<ul><li>Step1: Train learning algo.</li>
<li>Step2: Combiner algo is trained using algo predictions from step1. </li>
</ul>
    
    
</ul>
    

# II. Bagging (Bootstrap aggregating) #

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
import sys
import matplotlib

print('python:',sys.version)
print('Numpy:',np.__version__)
print('Pandas:',pd.__version__)
print('Scikitlearn:',sklearn.__version__)
print('Seaborn: ',sns.__version__)
print('matplotlib:',matplotlib.__version__)


In [ ]:
Titan=sns.load_dataset('titanic')
Titan.shape
Titan


In [ ]:
#Titan.dropna(inplace=True) #drop all NaN samples
Titan['pclass'].unique()
Titan['pclass'].value_counts()


In [ ]:
Titan['sex'].unique()
Titan['sex'].value_counts()


In [ ]:
Titan['age'].hist(bins=150)


## 1) Data pre-processing ##

In [ ]:
from sklearn import preprocessing

Titan2= Titan[['pclass', 'sex', 'age','alive']].copy()

LE=preprocessing.LabelEncoder()
#LE=preprocessing.LabelBinarizer()
Titan2['sex']=LE.fit_transform(Titan['sex'])
Titan2['alive']=LE.fit_transform(Titan['alive'])
Titan2


In [ ]:
# Impute NaN (missing) Values:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(Titan2)

Tit3=imp.transform(Titan2)
X=np.delete(Tit3,3,1)
Y=np.delete(Tit3,[0,1,2],1)
X, Y


In [ ]:
#obsolete part
from sklearn import preprocessing
X= Titan[['pclass', 'sex', 'age']].copy()
LE=preprocessing.LabelEncoder()
#LE=preprocessing.LabelBinarizer()
X['sex']=LE.fit_transform(Titan['sex'])
X.describe()
#X.info()
Y=Titan['survived']
Y.value_counts()

## 2) Fit Model: ##

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3)
#Y_test.describe()


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score

def print_score(clf, X_train, X_test, Y_train, Y_test, train=True):
    "Print the accuracy score, Classification report and confusion matrix"
    LB=preprocessing.LabelBinarizer()
    LB.fit(Y_train)
    if train:
        "Train performance"
        res=clf.predict(X_train)
        
        print(f'Train Results:\n')
        print(f'Accuracy Score: %.4f \n' % (accuracy_score(Y_train,res)))
        print(f'Classification report:\n {classification_report(Y_train,res)} ')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_train,res)}')
        print(f'ROC AUC Score:%.4f'%(roc_auc_score(LB.transform(Y_train),LB.transform(res))))
        
        res=cross_val_score(clf,X_train, Y_train, cv=10, scoring='accuracy')
        print('Average Accuracy \t: %.4f'%(np.mean(res)))
        print('Accuracy SD \t: %.4f '% (np.std(res)))
                      
    elif train==False:
        "Test performance"
        res_test=clf.predict(X_test)
        print(f'Test Results:\n')
        print('Accuracy Score: %.4f \n'%(accuracy_score(Y_test,res_test)))
        print(f'Classification report:\n {classification_report(Y_test,res_test)}')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_test,res_test)}')
        print('ROC AUC Score: %.4f\n'%(roc_auc_score(LB.transform(Y_test),LB.transform(res_test))))
        print('Average Accuracy \t: %.4f'%(np.mean(res_test)))
        print('Accuracy SD \t: %.4f '% (np.std(res_test)))


### a\ Decision Tree ###

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier(random_state=42)
clf.fit(X_train, Y_train)


In [ ]:
print_score(clf, X_train, X_test, Y_train, Y_test, train=True)


### b/ Bagging (oob_score=False): ###
Out of bag error (OOB): or out of bag estimate is a method of measuring the prediction error of random forests, boosted decision trees, and other ML models using bootstrap aggregating (bagging) to sub sample data samples used for training.<br>
OOB is the mean prediction error on each training sample $ x_i $, using only the tree that did not have  $ x_i $ in their bootstrap sample.<br>
Subsampling allows one to define an out of bag estimate of prediction performance improvement by evaluating predictions on those observations which were not used in the building of the next base learner.<br>
OOB estimate helps avoid the need for an independant validation dataset, but often underestimates actual performance improvement and the optimal number of iterations. 

In [ ]:
bag_clf=BaggingClassifier(estimator=clf, n_estimators=1000, bootstrap=True, 
                          oob_score=False, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, Y_train.ravel())


In [ ]:
print_score(bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)


 ### c/ Bagging (oob_score=True): ###

In [ ]:
bag_clf=BaggingClassifier(estimator=clf, n_estimators=1000, bootstrap=True, 
                          oob_score=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, Y_train.ravel())


In [ ]:
print_score(bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)


In [ ]:
print('OOB Score = %.4f'% (bag_clf.oob_score_))


 ### d/ Random Forest : ###

<ul>
<li>Ensemble of decision trees. </li>
<li>Training via the Bagging method(repeated sampling with replacement).</li>
<ul><li>Bagging: sample from samples. </li>
<li>RF: Sample from predictors: $ m=\sqrt(p) $ for classification and $ m=\frac{p}{3} $ for regression problems.</li>
</ul>
<li>Utilise uncorrelated trees
</ul>
<p>
Random Forest: sample both observations and features of training data.<br><br>
Bagging: <ul>
<li>Samples only observations at random.</li>
<li>Decision Tree select best feature when splitting a node.</li>
</ul>
</p>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

rf_clf=RandomForestClassifier(random_state=42, n_estimators=100)
rf_clf.fit(X_train, Y_train.ravel())


In [ ]:

print(f' {print_score(rf_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(rf_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


 ### e/ Grid Search : ###

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

param_grid={'max_depth':[3,None], 'min_samples_split':[2,3,10],
           'min_samples_leaf':[1,3,10] , 'bootstrap':[True, False] ,
           'criterion':['gini', 'entropy']}
grid_serc=GridSearchCV(rf_clf,param_grid,n_jobs=-1, cv=5, 
                       verbose=1, scoring='accuracy', return_train_score=False)
grid_serc.fit(X_train, Y_train.ravel())


In [ ]:

print(f' {print_score(grid_serc, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(grid_serc, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


In [ ]:
grid_serc.best_score_
grid_serc.best_estimator_.get_params()


 ### f/ Extra Trees Ensemble (Extremely Randomized Trees) : ###
 <ul><li>Random forest is build upon Decision Tree.</li>
 <li>Decision tree node splitting is based on gini or entropy or some other algorithms.</li>
 <li>Extra Trees make use of random thresholds for each feature unlike decision tree.</li>
</ul>


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

xt_clf=ExtraTreesClassifier(random_state=42, n_estimators=100)
xt_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


# III. Boosting: #
<br>Combine several weak learners into a strong learner.<br>
Train predictors sequentially

## 1) AdaBoost (Adaptive Boosting) (CART): ##
<ul><li>Similar to human learning, the algorithm learns from past mistakes by focusing more on difficult problems it did not get right in prior learning.</li>
<li>In machine learning language: it pays more attention to training instances that previously underfitted.</li>
</ul>


in Scikit-learn:
<ul>
<li>Fit a sequence of weak learners (models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of data.</li>
<li>The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction.</li>
<li>The data modifications at each so called boosting iteration consist in applying weights w1,w2...wn to each of the training samples.</li>
<li>Initially, those weights are all set to $w_i=\frac {1}{N}$, so that the 1st step simply trains a weak learner on the original data.</li>
<li>For each successive iteration, the sample weights are individually modified and the learning algorithm is reapplied to the reweighted data.   </li>
<li>At a given step, those training examples that were incorreclty predicted by the boosted model induced at the previous step have their weights increased, whereas the weights are decreased for those that were predicted correctly. </li>
<li>As iterations proceed, examples that are difficult to predict receive ever-increasing influence, each subsequent weak learner is thereby forced to cencentrate on the examples that are missed by the previous ones in the sequence. </li>

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf=AdaBoostClassifier(n_estimators=100, random_state=42)
ada_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 2) AdaBoost with Random Forest: ##

In [ ]:
from sklearn.ensemble import RandomForestClassifier

ada_clf=AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
ada_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 3) Gradient Boosting Machine (GBM): ##

works with regression and classification
<ul><li>Sequentially adding predictors</li>
<li>Each one correcting its predecessor</li>
<li>Fit a new predictor to the residual errors</li></ul>
Compare this to AdaBoost:
Alter instance weights at every iteration:
<ul>
<li>Step 1: $ Y=F(x)+\epsilon $ </li>
<li>Step 2: $ \epsilon=G(x)+\epsilon_2 $ </li>
<li>Step 3: $ Y=F(x)+G(x)+\epsilon_2 $ </li>
<li>Step 4: $ \epsilon_2=H(x)+\epsilon_3 $ </li>
<li>Step 5: $ Y=F(x)+G(x)+H(x)+\epsilon_3 $ </li>
<li>Step finale: $ Y=\alpha F(x)+\beta G(x)+\gamma H(x)+\epsilon_4 $ </li>
</ul>
Gradient boosting involve 3 elements:
<ul>
<li>Loss function to be optimized: loss function depends on the type of problem being solved. In the case of regression problems, MSE is used. In classification problems, logarithmic loss will be used. In boosting, at each stage, unexplained loss from prior iterations will be optimized rather than starting from scratch. </li>
<li>Weak learner to make predictions: Decision trees are used as weak learner in gradient boosting. </li>
<li>Additive model to add weak learners to minimize the loss function: Trees are added one at a time and existing trees in the model are not changed. The gradient descent procedure is used to minimize the loss when adding trees. </li>
</ul>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc_clf=GradientBoostingClassifier()
gbc_clf.fit(X_train,Y_train.ravel())


In [ ]:
print(f' {print_score(gbc_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(gbc_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 4) XGBoost(Extreme Gradient Boosting) : ##

### Woking Test ###

In [ ]:
import xgboost as xgb

data=np.random.rand(100,10)
label=np.random.randint(2, size=100)
dtrain=xgb.DMatrix(data, label=label)
dtrain


In [ ]:
dtest=dtrain
param={'bst:max_depth':2, 'bst_eta':1, 'silent':1, 'objective':'binary:logistic'}
param['nthread']=4
param['eval_metric']='auc'
evallist=[(dtest,'eval'),(dtrain,'train')]


In [ ]:
num_round=10
bst=xgb.train(param, dtrain, num_round,evallist )


### objective function: training loss + Regularization ###
$ obj(\Theta)=L(\theta)+ \gamma (\Theta) $
<ul>
<li> L is a training loss function </li>
<li> $\gamma $ is regularization term </li>
</ul>
</p>

### Training Loss: ###
It measures how predictve our model is on training data<br>
Example1: MSE on linear regression: <br>
$ L(\theta)=\Sigma_i (y_i-\hat{y}_i)^2 $ <br>
Example2: Logistic loss for logistic regression: <br>
$ L(\theta)=\Sigma_i [(y_iln(1+e^{-\hat{y}_i})+ (1-y_i)ln(1+e^{\hat{y}_i})] $
### Regularization term: ###
The regularization term controlls the complexity of the model, which helps us to avoid overfitting


--------------------------------------
### Underfitting ###

A statistical model or a machine learning algorithm is said to have underfitting when it cannot capture the underlying trend of the data, i.e., it only performs well on training data but performs poorly on testing data. <br>
Reasons for Underfitting:
<ul>
<li>    High bias and low variance. </li>
<li>    The size of the training dataset used is not enough. </li>
<li>    The model is too simple. </li>
<li>    Training data is not cleaned and also contains noise in it. </li>
</ul>
Techniques to Reduce Underfitting
<ul>
<li>     Increase model complexity. </li>
<li>     Increase the number of features, performing feature engineering. </li>
<li>     Remove noise from the data. </li>
<li>     Increase the number of epochs or increase the duration of training to get better results. </li>
</ul>

### Overfitting ###
A statistical model is said to be overfitted when the model does not make accurate predictions on testing data. When a model gets trained with so much data, it starts learning from the noise and inaccurate data entries in our data set. And when testing with test data results in High variance. <br>
Reasons for Overfitting:
<ul>
<li>     High variance and low bias. </li>
<li>    The model is too complex. </li>
<li>    The size of the training data. </li>
</ul>
Techniques to Reduce Overfitting
<ul>
<li>    Increase training data. </li>
<li>    Reduce model complexity. </li>
<li>    Early stopping during the training phase (have an eye over the loss over the training period as soon as loss begins to increase stop training). </li>
<li>    Ridge Regularization and Lasso Regularization. </li>
<li>    Use dropout for neural networks to tackle overfitting. </li>
</ul>


### XGBoost VS GBM: ###
<ul>
<li>Specially, XGBoost is uses a more regularized model formalization to control overfitting, which gives it better performance.</li>
<li>For model, it might be more suitable to be called as regularized gradient boosting. </li>
</ul>


In [ ]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(max_depth=5, n_estimators=10000, learning_rate=0.3, n_jobs=-1)
xgb_clf.fit(X_train, Y_train.ravel())


In [ ]:

print(f' {print_score(xgb_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(xgb_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


# IV. HR Project: #
Employee attrition is defined as employees leaving their organizations for unpredictable or uncontrollable reasons.

In [ ]:
HR=pd.read_csv(r'Data/HR-Employee-Attrition.csv')
HR


In [ ]:
num_col=list(HR.describe().columns)
categorical_col=list(set(HR.columns).difference(num_col))
remove_list=['EmplyeeCount','EmplyeeNumber','StandardHours']
col_numerical=[e for e in num_col if e not in remove_list]
categorical_col.remove('Attrition')
#categorical_col
#col_numerical.remove('Attrition_num')
col_numerical


In [ ]:

Attrition_num={'Yes':1, 'No':0}
HR['Attrition_num']=HR['Attrition'].map(Attrition_num)

DF_categ=pd.get_dummies(HR[categorical_col])
X=pd.concat([HR[col_numerical], DF_categ], axis=1)
Y=HR['Attrition_num']
X


In [ ]:
#HR[col_numerical].corr()
plt.figure(figsize=(20, 10))
sns.heatmap(HR[col_numerical].corr(), annot=True, fmt='.3f')


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test=train_test_split(X,Y,random_state=1301,test_size=0.3)
#Y_test.describe()


In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score

def print_score(clf, X_train, X_test, Y_train, Y_test, train=True):
    "Print the accuracy score, Classification report and confusion matrix"
    LB=preprocessing.LabelBinarizer()
    LB.fit(Y_train)
    if train:
        "Train performance"
        res=clf.predict(X_train)
        
        print(f'Train Results:\n')
        print(f'Accuracy Score: %.4f \n' % (accuracy_score(Y_train,res)))
        print(f'Classification report:\n {classification_report(Y_train,res)} ')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_train,res)}')
        print(f'ROC AUC Score:%.4f'%(roc_auc_score(LB.transform(Y_train),LB.transform(res))))
        
        res=cross_val_score(clf,X_train, Y_train, cv=10, scoring='accuracy')
        print('Cross Val. average accuracy \t: %.4f'%(np.mean(res)))
        print('Cross Val. accuracy SD \t: %.4f '% (np.std(res)))
                      
    elif train==False:
        "Test performance"
        res_test=clf.predict(X_test)
        print(f'Test Results:\n')
        print('Accuracy Score: %.4f \n'%(accuracy_score(Y_test,res_test)))
        print(f'Classification report:\n {classification_report(Y_test,res_test)}')
        print(f'Confusion Matrix Score: {confusion_matrix(Y_test,res_test)}')
        print('ROC AUC Score: %.4f\n'%(roc_auc_score(LB.transform(Y_test),LB.transform(res_test))))
        print('Cross Val. average accuracy \t: %.4f'%(np.mean(res_test)))
        print('Cross Val. average accuracy SD \t: %.4f '% (np.std(res_test)))


## 1) Decision Tree : ##

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT_clf=DecisionTreeClassifier(random_state=42)
DT_clf.fit(X_train, Y_train)


In [ ]:
print(f' {print_score(DT_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(DT_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 2) Bagging : ##

In [ ]:
from sklearn.ensemble import BaggingClassifier

Bag_clf=BaggingClassifier(estimator=DT_clf ,n_estimators=100, bootstrap=True,
                          n_jobs=-1, random_state=42)
Bag_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(Bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(Bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 3) Random Forest : ##

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF_clf=RandomForestClassifier(n_estimators=100)
RF_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


In [ ]:

pd.Series(RF_clf.feature_importances_, index=X_train.columns).sort_values(ascending=False).plot(kind='bar',figsize=(12,6))


the use of a forest of trees to evaluate the importance of features on an artificial classification task. The blue bars are the feature importances of the forest, along with their inter-trees variability represented by the error bars(not shown in this case).

## 4) AdaBoost : ##

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

Ada_clf=AdaBoostClassifier()
Ada_clf.fit(X_train, Y_train.ravel())

In [ ]:
print(f' {print_score(Ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(Ada_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 5) AdaBoost + Random Forest: ##

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

Ada_RF=AdaBoostClassifier(RandomForestClassifier(n_estimators=100), n_estimators=100)
Ada_RF.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(Ada_RF, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(Ada_RF, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 6) Gradient Boosting Classifier : ##

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc_clf=GradientBoostingClassifier(n_estimators=100)
gbc_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(gbc_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(gbc_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 7) XG Boost : ##

In [ ]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(n_estimators=100)
xgb_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(xgb_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(xgb_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


# V. Ensemble of ensembles - Model stacking : #


<b>Ensemble with different types of classifiers:</b>
<ul>
<li>Different type of classifiers (logistic regression, decision trees, random forest,...) are fitted on the same training data</li>
<li>results are combined based on either:<br>
majority voting (classification)<br>
average (regression)</li>
</ul>
<b>Ensemble with a single type of classifier:</b>
<ul>
<li>Bootstrap samples are drawn with training data.</li>
<li>With each bootstrap sample , model (individual model maybe decision trees or random forest) will be fitted.</li>
<li>All the results are combined to create an ensemble.</li>
<li>Suitable for highly flexible models that are prone to overfitting / high variance. </li>
</ul>
    

## Combining Method ##
<ul>
<li><b>Majority voting or average:</b><br>
Classification: largest number of votes (mode)<br>
Regression problems: Average (mean) 
</li>
<li><b>Method of application of meta-classifiers on outcomes: </b><br>
Binary outcomes: 0/1 from individual classifiers.<br>
Meta classifier is applied on top of the individual classifiers.
<li><b>Method of application of meta-classifiers on probabilities:</b><br>
Probabilities are obtained from individual classifiers.<br>
Applying meta-classifier.


## 1) Model1: Decision Tree. ##

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT_clf=DecisionTreeClassifier(random_state=42)
DT_clf.fit(X_train, Y_train)


In [ ]:
print(f' {print_score(DT_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(DT_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


## 2) Model2: Random Forest. ##

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF_clf=RandomForestClassifier(n_estimators=100)
RF_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(RF_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


In [ ]:
en_en=pd.DataFrame()
DT_clf.predict_proba(X_train)


In [ ]:
en_en['DT_clf']= pd.DataFrame(DT_clf.predict_proba(X_train))[1]
en_en['RF_clf']= pd.DataFrame(RF_clf.predict_proba(X_train))[1]
col_name=en_en.columns
#en_en['ind']=pd.DataFrame(Y_train)
en_en=pd.concat([en_en['DT_clf'],en_en['RF_clf'],pd.DataFrame(Y_train).reset_index(drop=True)], axis=1)
en_en


## 3) Meta Classifier. ## 

In [ ]:
from sklearn.linear_model import LogisticRegression

m_clf=LogisticRegression(fit_intercept=False, solver='lbfgs')
Xm=en_en.drop(['Attrition_num'], axis=1)
Ym=en_en['Attrition_num']
Xm.dropna(inplace=True)
Ym.dropna(inplace=True)
m_clf.fit(Xm,Ym)


In [ ]:
en_test=pd.DataFrame()
en_test['DT_clf']= pd.DataFrame(DT_clf.predict_proba(X_test))[1]
en_test['RF_clf']= pd.DataFrame(RF_clf.predict_proba(X_test))[1]
en_test['Combined']=m_clf.predict(en_test[['DT_clf','RF_clf']])
en_test=pd.concat([en_test['DT_clf'],en_test['RF_clf'],en_test['Combined'],pd.DataFrame(Y_test).reset_index(drop=True)],
                axis=1)
en_test


In [ ]:
print(pd.crosstab(en_test['Attrition_num'],en_test['Combined']))


In [ ]:
print(round(accuracy_score(en_test['Attrition_num'],en_test['Combined']),4))


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(en_test['Attrition_num'],en_test['Combined']))


## 3) Single Classifier. ## 

In [ ]:
X


In [ ]:
HR['Attrition'].value_counts() / HR['Attrition'].count()


In [ ]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier

class_weight={0:0.838776 , 1:0.161224}
pd.Series(list(Y_train)).value_counts()/pd.Series(list(Y_train)).count()


In [ ]:
forest=RandomForestClassifier(class_weight=class_weight, n_estimators=100)


In [ ]:
Ada=AdaBoostClassifier(estimator=forest, n_estimators=100, learning_rate=0.5, random_state=42)
Ada.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(Ada, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(Ada, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')


In [ ]:
Bag_clf=BaggingClassifier(estimator=Ada, n_estimators=50, max_samples=1.0, max_features=1.0,
                          bootstrap=True, bootstrap_features=False, n_jobs=-1,
                          random_state=42)
Bag_clf.fit(X_train, Y_train.ravel())


In [ ]:
print(f' {print_score(Bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=True)}')
print(f'\n ---------------------------------------\n')
print(f'{print_score(Bag_clf, X_train, X_test, Y_train.ravel(), Y_test.ravel(), train=False)}')
